# 🌊 Structured Streaming Basics: Real-Time Data Processing

**Time to complete:** 40 minutes  
**Difficulty:** Intermediate  
**Prerequisites:** DataFrames, SQL, basic streaming concepts

---

## 🎯 Learning Objectives

By the end of this notebook, you will master:
- ✅ **Streaming fundamentals** - Unbounded vs bounded data
- ✅ **Structured Streaming concepts** - DataFrames for streams
- ✅ **Basic operations** - Reading, transforming, writing streams
- ✅ **Output modes** - Append, Update, Complete modes
- ✅ **Triggers** - Processing time vs event time
- ✅ **Fault tolerance** - Checkpointing and recovery
- ✅ **Monitoring** - Streaming query statistics

**Structured Streaming brings DataFrame power to real-time processing!**

---

## 🔍 Understanding Streaming Fundamentals

### Bounded vs Unbounded Data

**Traditional batch processing** works with bounded data:
```
Batch Processing:
┌─────────────────┐     ┌──────────────┐     ┌──────────────┐
│   Input Data    │────▶│  Processing  │────▶│  Output      │
│   (Fixed Size)  │     │  (One Pass)  │     │  (Complete)  │
└─────────────────┘     └──────────────┘     └──────────────┘
         ↑                     ↑                     ↑
     Static Data          Finite Time         Final Result
```

**Streaming processing** handles unbounded data:
```
Stream Processing:
┌─────────────────┐     ┌──────────────┐     ┌──────────────┐
│   Input Stream  │────▶│  Continuous  │────▶│  Continuous  │
│   (Never Ends)  │     │  Processing  │     │  Output      │
└─────────────────┘     └──────────────┘     └──────────────┘
         ↑                     ↑                     ↑
   Continuous Data      Infinite Time      Evolving Results
```

**Key differences:**
- **Bounded**: Fixed dataset, one-time processing
- **Unbounded**: Continuous data, ongoing processing
- **Streaming challenge**: Handle late data, state management, fault tolerance

### Structured Streaming Architecture

**Structured Streaming** treats streams as unbounded DataFrames:
```
Streaming DataFrame:
┌─────────────────┐     ┌─────────────────┐     ┌─────────────────┐
│   Input Stream  │────▶│   DataFrame     │────▶│   Output Sink   │
│   (Kafka/Socket)│     │   Operations    │     │   (Console/File)│
└─────────────────┘     └─────────────────┘     └─────────────────┘
         │                       │                       │
     Source                Transformations           Sink
```

**Same DataFrame API** - streaming and batch use identical operations!

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, window, count, sum as sum_func
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType
import pyspark.sql.functions as F
import time
import threading

spark = SparkSession.builder \
    .appName("Structured_Streaming_Basics") \
    .master("local[*]") \
    .getOrCreate()

print(f"✅ Spark ready - Version: {spark.version}")

# Enable adaptive query execution for streaming
spark.conf.set("spark.sql.adaptive.enabled", "true")
spark.conf.set("spark.sql.streaming.forceDeleteTempCheckpointLocation", "true")

print("Structured Streaming environment configured")


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/12/20 12:12:04 WARN Utils: Your hostname, Devs-MacBook-Air.local, resolves to a loopback address: 127.0.0.1; using 192.168.0.107 instead (on interface en0)
25/12/20 12:12:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/20 12:12:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


✅ Spark ready - Version: 4.1.0
Structured Streaming environment configured


## 📡 Creating Your First Streaming Query

### Rate Source (For Learning)

**Rate source** generates test data automatically:

In [ ]:
# Create a streaming DataFrame from rate source
print("📡 CREATING FIRST STREAMING QUERY")
print("=" * 50)

# Rate source generates rows with timestamp and value
streaming_df = spark.readStream \
    .format("rate") \
    .option("rowsPerSecond", 1) \
    .load()

print("Streaming DataFrame created:")
streaming_df.printSchema()
print(f"Is streaming: {streaming_df.isStreaming}")

# Show sample data (limited for demo)
print("\nSample streaming data:")
try:
    # Convert to static DataFrame for demonstration
    sample_df = streaming_df.limit(5)
    sample_static = spark.createDataFrame(sample_df.collect())
    sample_static.show()
except:
    print("Rate source - data will be generated when query starts")

# Basic transformation (same as DataFrame API)
transformed_df = streaming_df \
    .withColumn("timestamp_str", col("timestamp").cast("string")) \
    .withColumn("value_doubled", col("value") * 2) \
    .filter(col("value") > 5)

print("\nTransformed streaming DataFrame:")
transformed_df.printSchema()


### Writing Streaming Results

**Streaming queries write results continuously:**

In [ ]:
# Write streaming results to console
print("📝 STREAMING QUERY OUTPUT")
print("=" * 50)

# Create console output query
query = transformed_df.writeStream \
    .outputMode("append") \
    .format("console") \
    .option("truncate", "false") \
    .trigger(processingTime="5 seconds") \
    .start()

print("Streaming query started!")
print(f"Query ID: {query.id}")
print(f"Query name: {query.name}")
print(f"Is active: {query.isActive}")

# Let it run for a few seconds
time.sleep(15)

# Stop the query
query.stop()
print("\nQuery stopped")


### Understanding Output Modes

**Output modes** determine how results are written:

In [ ]:
# Demonstrate different output modes
print("📊 OUTPUT MODES EXPLAINED")
print("=" * 50)

# Create a simple aggregation for demonstration
aggregated_df = streaming_df \
    .groupBy() \
    .agg(
        count("*").alias("total_count"),
        sum_func("value").alias("total_sum"),
        F.avg("value").alias("avg_value")
    )

print("Aggregated streaming DataFrame:")
aggregated_df.printSchema()

# Output mode comparison
output_modes = {
    "complete": "All rows in result table (for global aggregations)",
    "append": "Only new rows since last trigger (for append-only operations)",
    "update": "Only changed rows since last trigger (for upserts)"
}

print("\n📋 OUTPUT MODE COMPARISON:")
for mode, description in output_modes.items():
    print(f"\n{mode.upper()} MODE:")
    print(f"  Description: {description}")

    # When to use
    if mode == "complete":
        print("  Use for: Global aggregations, running totals")
        print("  Example: Total sales, global statistics")
    elif mode == "append":
        print("  Use for: Append-only operations, map operations")
        print("  Example: Event processing, filtering")
    elif mode == "update":
        print("  Use for: Upserts, changing aggregations")
        print("  Example: Windowed aggregations, deduplication")

# Demonstrate complete mode
print("\n🚀 DEMONSTRATING COMPLETE MODE:")
complete_query = aggregated_df.writeStream \
    .outputMode("complete") \
    .format("console") \
    .option("truncate", "false") \
    .trigger(processingTime="3 seconds") \
    .start()

time.sleep(12)  # Let it run for 12 seconds
complete_query.stop()
print("Complete mode demonstration finished")


## ⏰ Triggers: When to Process Data

### Processing Time vs Event Time

**Triggers** control when streaming queries process data:

In [ ]:
# Trigger types demonstration
print("⏰ STREAMING TRIGGERS")
print("=" * 50)

# Trigger types
trigger_types = {
    "processingTime": "Process based on wall-clock time",
    "once": "Process all available data once, then stop",
    "continuous": "Process data as soon as it arrives (experimental)"
}

print("TRIGGER TYPES:")
for trigger_type, description in trigger_types.items():
    print(f"\n{trigger_type.upper()}:")
    print(f"  {description}")

# Demonstrate different triggers
print("\n🚀 TRIGGER DEMONSTRATIONS:")

# 1. Processing time trigger (most common)
print("\n1. PROCESSING TIME TRIGGER (every 2 seconds):")
pt_query = streaming_df.writeStream \
    .outputMode("append") \
    .format("console") \
    .trigger(processingTime="2 seconds") \
    .start()

time.sleep(8)
pt_query.stop()
print("Processing time trigger stopped")

# 2. Once trigger (batch-like)
print("\n2. ONCE TRIGGER (process available data once):")
once_query = streaming_df.limit(10).writeStream \
    .outputMode("append") \
    .format("console") \
    .trigger(once=True) \
    .start()

once_query.awaitTermination(timeout=10)  # Wait for completion
print("Once trigger completed")

# 3. Default trigger (as fast as possible)
print("\n3. DEFAULT TRIGGER (as fast as possible):")
default_query = streaming_df.writeStream \
    .outputMode("append") \
    .format("console") \
    .start()  # No trigger specified = default

time.sleep(6)
default_query.stop()
print("Default trigger stopped")


## 💾 Fault Tolerance and Checkpointing

### Checkpointing for Reliability

**Checkpointing** saves streaming query state for fault recovery:

In [ ]:
# Checkpointing for fault tolerance
print("💾 FAULT TOLERANCE & CHECKPOINTING")
print("=" * 50)

# Create checkpoint directory
import os
checkpoint_dir = "/tmp/streaming_checkpoint"
if os.path.exists(checkpoint_dir):
    import shutil
    shutil.rmtree(checkpoint_dir)

print(f"Using checkpoint directory: {checkpoint_dir}")

# Streaming query with checkpointing
checkpointed_query = streaming_df \
    .withColumn("running_count", count("*").over(
        window(windowDuration="30 seconds", slideDuration="10 seconds")
    )) \
    .writeStream \
    .outputMode("update") \
    .format("console") \
    .option("checkpointLocation", checkpoint_dir) \
    .trigger(processingTime="3 seconds") \
    .start()

print("Checkpointed streaming query started")
print("Checkpoints save:")
print("- Query metadata and configuration")
print("- Current batch ID and offsets")
print("- Aggregated state for stateful operations")

time.sleep(12)
checkpointed_query.stop()
print("\nCheckpointed query stopped")

# Check checkpoint directory
if os.path.exists(checkpoint_dir):
    print(f"\nCheckpoint files created in {checkpoint_dir}:")
    for root, dirs, files in os.walk(checkpoint_dir):
        level = root.replace(checkpoint_dir, '').count(os.sep)
        indent = ' ' * 2 * level
        print(f"{indent}{os.path.basename(root)}/")
        subindent = ' ' * 2 * (level + 1)
        for file in files:
            print(f"{subindent}{file}")
else:
    print("Checkpoint directory not found")


## 📊 Monitoring Streaming Queries

### Query Statistics and Management

In [ ]:
# Monitoring streaming queries
print("📊 STREAMING QUERY MONITORING")
print("=" * 50)

# Start a query to monitor
monitor_query = streaming_df \
    .groupBy(window("timestamp", "10 seconds")) \
    .count() \
    .writeStream \
    .outputMode("update") \
    .format("console") \
    .option("truncate", "false") \
    .trigger(processingTime="5 seconds") \
    .start()

print("Monitoring streaming query...")

# Monitor query status
def print_query_status(query):
    print(f"\nQuery Status:")
    print(f"  ID: {query.id}")
    print(f"  Name: {query.name}")
    print(f"  Active: {query.isActive}")

    if hasattr(query, 'lastProgress'):
        progress = query.lastProgress
        if progress:
            print(f"  Batch ID: {progress.get('batchId', 'N/A')}")
            print(f"  Input Rows: {progress.get('numInputRows', 0)}")
            print(f"  Processed Rows: {progress.get('processedRowsPerSecond', 0):.1f}/sec")
            print(f"  Batch Duration: {progress.get('batchDuration', 'N/A')}ms")

# Monitor for a few batches
for i in range(3):
    time.sleep(6)  # Wait for batch to complete
    print_query_status(monitor_query)

monitor_query.stop()
print("\nMonitoring completed")

# List all active streaming queries
print("\nActive streaming queries:")
for q in spark.streams.active:
    print(f"- {q.name} ({q.id}): {'Active' if q.isActive else 'Inactive'}")

if not spark.streams.active:
    print("No active streaming queries")


## 🔧 Advanced Streaming Concepts

### Handling Different Data Sources

In [ ]:
# Advanced streaming concepts
print("🔧 ADVANCED STREAMING CONCEPTS")
print("=" * 50)

# 1. Multiple streaming sources
print("1. MULTIPLE STREAMING SOURCES")

# Create two different rate streams
stream1 = spark.readStream.format("rate").option("rowsPerSecond", 2).load()
stream2 = spark.readStream.format("rate").option("rowsPerSecond", 1).load()

# Add source identifier
stream1_labeled = stream1.withColumn("source", F.lit("stream1"))
stream2_labeled = stream2.withColumn("source", F.lit("stream2"))

# Union streams
combined_stream = stream1_labeled.union(stream2_labeled)

print("Combined streaming sources:")
combined_stream.printSchema()

# 2. Streaming with static data joins
print("\n2. STREAMING WITH STATIC DATA JOINS")

# Create static lookup table
lookup_data = [
    (0, "Even"),
    (1, "Odd"),
    (2, "Even"),
    (3, "Odd")
]

lookup_df = spark.createDataFrame(lookup_data, ["value", "parity"])

# Join streaming data with static lookup
enriched_stream = combined_stream.join(
    F.broadcast(lookup_df),
    combined_stream["value"] == lookup_df["value"],
    "left"
)

print("Streaming + static data join:")
enriched_stream.printSchema()

# 3. Conditional streaming logic
print("\n3. CONDITIONAL STREAMING LOGIC")

conditional_stream = enriched_stream.withColumn(
    "status",
    F.when(col("value") > 5, "High")
     .when(col("value") > 2, "Medium")
     .otherwise("Low")
).withColumn(
    "alert",
    F.when(col("source") == "stream1", "Priority")
     .otherwise("Normal")
)

# Demonstrate complex streaming pipeline
complex_query = conditional_stream.writeStream \
    .outputMode("append") \
    .format("console") \
    .option("truncate", "false") \
    .trigger(processingTime="4 seconds") \
    .start()

time.sleep(16)
complex_query.stop()
print("\nComplex streaming pipeline completed")


## 🚨 Error Handling and Recovery

### Handling Streaming Failures

In [ ]:
# Error handling and recovery
print("🚨 ERROR HANDLING AND RECOVERY")
print("=" * 50)

# 1. Graceful shutdown
print("1. GRACEFUL SHUTDOWN")

def graceful_shutdown(query, timeout_seconds=30):
    """Gracefully shutdown streaming query"""
    try:
        print(f"Initiating graceful shutdown for query {query.id}...")

        # Stop accepting new data
        query.stop()

        # Wait for processing to complete
        query.awaitTermination(timeout_seconds * 1000)

        print("✅ Query shutdown successfully")

    except Exception as e:
        print(f"⚠️  Error during shutdown: {e}")
        # Force stop if graceful shutdown fails
        query.stop()

# 2. Recovery from checkpoint
print("\n2. RECOVERY FROM CHECKPOINT")

def recover_or_create_query(checkpoint_path, create_query_func):
    """Recover query from checkpoint or create new one"""

    if os.path.exists(checkpoint_path):
        print(f"Found checkpoint at {checkpoint_path}")
        print("Attempting to recover query...")

        try:
            # In production, you'd recover the specific query
            # For demo, we'll create a new one
            query = create_query_func()
            print("✅ Query recovered successfully")
            return query

        except Exception as e:
            print(f"❌ Recovery failed: {e}")
            print("Creating new query...")

    # Create new query
    query = create_query_func()
    print("✅ New query created")
    return query

# 3. Monitoring and alerting
print("\n3. MONITORING AND ALERTING")

def monitor_streaming_health(query, alert_threshold_seconds=60):
    """Monitor streaming query health"""

    last_batch_time = time.time()

    while query.isActive:
        time.sleep(10)  # Check every 10 seconds

        current_time = time.time()

        # Check if query is still processing
        if hasattr(query, 'lastProgress') and query.lastProgress:
            progress = query.lastProgress
            batch_id = progress.get('batchId', 0)

            # Update last batch time
            if batch_id > 0:
                last_batch_time = current_time

        # Alert if no batches processed recently
        time_since_last_batch = current_time - last_batch_time
        if time_since_last_batch > alert_threshold_seconds:
            print(f"🚨 ALERT: No batch processed in {time_since_last_batch:.0f} seconds")
            print("Query may be stuck or failing")
            break

    print("Health monitoring stopped")

# Demonstrate error handling
print("\nDEMONSTRATING ERROR HANDLING:")

# Create a query that might fail
test_query = streaming_df.writeStream \
    .outputMode("append") \
    .format("console") \
    .trigger(processingTime="2 seconds") \
    .start()

# Start health monitoring in background
monitor_thread = threading.Thread(target=monitor_streaming_health, args=(test_query,))
monitor_thread.daemon = True
monitor_thread.start()

# Let it run for a bit
time.sleep(20)

# Graceful shutdown
graceful_shutdown(test_query, 10)

print("\nError handling demonstration completed")


## 🎯 Streaming Best Practices

### Production-Ready Streaming

In [ ]:
# Streaming best practices
print("🎯 STREAMING BEST PRACTICES")
print("=" * 50)

# 1. Resource configuration
print("1. RESOURCE CONFIGURATION")
streaming_configs = {
    "spark.streaming.backpressure.enabled": "true",
    "spark.streaming.receiver.maxRate": "10000",
    "spark.streaming.kafka.maxRatePerPartition": "1000",
    "spark.sql.shuffle.partitions": "50",
    "spark.streaming.concurrentJobs": "3"
}

print("Recommended streaming configurations:")
for key, value in streaming_configs.items():
    print(f"  {key} = {value}")

# 2. Checkpointing strategy
print("\n2. CHECKPOINTING STRATEGY")
checkpoint_best_practices = [
    "Use reliable storage (HDFS/S3) for checkpoints",
    "Configure appropriate checkpoint intervals",
    "Monitor checkpoint size and cleanup old checkpoints",
    "Test recovery from checkpoints regularly",
    "Use separate checkpoint locations for different queries"
]

for practice in checkpoint_best_practices:
    print(f"  ✓ {practice}")

# 3. Monitoring strategy
print("\n3. MONITORING STRATEGY")
monitoring_best_practices = [
    "Monitor batch processing time and throughput",
    "Set up alerts for query failures and delays",
    "Track input data rate and backpressure",
    "Monitor checkpoint and state store sizes",
    "Log and analyze query progress metrics"
]

for practice in monitoring_best_practices:
    print(f"  ✓ {practice}")

# 4. Error handling strategy
print("\n4. ERROR HANDLING STRATEGY")
error_handling = [
    "Implement graceful shutdown procedures",
    "Use checkpointing for automatic recovery",
    "Handle deserialization errors gracefully",
    "Implement dead letter queues for bad records",
    "Set appropriate timeouts and retry policies"
]

for practice in error_handling:
    print(f"  ✓ {practice}")

# 5. Performance optimization
print("\n5. PERFORMANCE OPTIMIZATION")
performance_tips = [
    "Use appropriate output modes for your use case",
    "Minimize state in stateful operations",
    "Use watermarking to limit state growth",
    "Choose efficient data formats (Parquet/ORC)",
    "Optimize trigger intervals based on latency requirements"
]

for tip in performance_tips:
    print(f"  ✓ {tip}")

print("\n🎉 STREAMING BEST PRACTICES SUMMARY:")
print("- Start simple, then optimize based on requirements")
print("- Always use checkpointing in production")
print("- Monitor query performance continuously")
print("- Test failure scenarios and recovery procedures")
print("- Balance throughput vs latency based on use case")


## 🎯 Key Takeaways

### Streaming Fundamentals Learned:
- ✅ **Unbounded data** - Continuous processing vs batch
- ✅ **Structured Streaming** - DataFrame API for streams
- ✅ **Output modes** - Append, Update, Complete semantics
- ✅ **Triggers** - Processing time vs event time control
- ✅ **Checkpointing** - Fault tolerance and state recovery
- ✅ **Monitoring** - Query statistics and health checks

### Core Streaming Concepts:
- 🔸 **Source**: Where streaming data comes from (Kafka, files, sockets)
- 🔸 **Processing**: Same DataFrame operations as batch
- 🔸 **Sink**: Where results are written (files, databases, consoles)
- 🔸 **State**: Maintained across batches for aggregations
- 🔸 **Checkpointing**: Saves progress for fault recovery

### Key Differences from Batch:
- 🔸 **Continuous**: Never-ending data processing
- 🔸 **Incremental**: Process data as it arrives
- 🔸 **Stateful**: Maintain state across time windows
- 🔸 **Fault-tolerant**: Automatic recovery from failures
- 🔸 **Late data**: Handle out-of-order events

### Output Mode Selection:
- 🔸 **Append**: New rows only (map operations, filtering)
- 🔸 **Complete**: All rows (global aggregations)
- 🔸 **Update**: Changed rows only (windowed aggregations)

### Trigger Types:
- 🔸 **Processing Time**: Wall-clock based (most common)
- 🔸 **Once**: Process available data once (batch-like)
- 🔸 **Continuous**: As-fast-as-possible (experimental)

### Production Considerations:
- 🔸 **Checkpointing**: Essential for fault tolerance
- 🔸 **Monitoring**: Track query health and performance
- 🔸 **Resource tuning**: Memory, CPU, parallelism
- 🔸 **Error handling**: Graceful failure recovery
- 🔸 **Backpressure**: Handle data rate mismatches

---

## 🚀 Next Steps

Now that you understand streaming basics, you're ready for:

1. **Windowing Operations** - Time-based aggregations
2. **Event Time Processing** - Handling late and out-of-order data
3. **Watermarking** - Managing state in streaming queries
4. **Stateful Operations** - Advanced streaming transformations
5. **Streaming Joins** - Joining streaming data
6. **Sources and Sinks** - Kafka, files, databases

**Structured Streaming makes real-time data processing as easy as batch processing!**

---

**🎉 Congratulations! You now have streaming superpowers in Spark!**